In [2]:
#!pip install -q transformers torch
!pip install datasets

!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install rouge_score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 6.9 MB/s 
     |████████████████████████████████| 115 kB 61.1 MB/s 
     |████████████████████████████████| 182 kB 86.2 MB/s 
     |████████████████████████████████| 212 kB 91.3 MB/s 
     |████████████████████████████████| 127 kB 88.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.3 MB/s 
     |████████████████████████████████| 1.3 MB 63.6 MB/s 
     |████████████████████████████████| 7.6 MB 44.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=

In [3]:
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict

from datasets import load_dataset
import pandas as pd
import torch
from transformers.tokenization_utils_base import BatchEncoding
from typing import List, Tuple
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_metric
import tensorflow as tf
import os
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from nltk.tokenize import sent_tokenize
import nltk
import evaluate
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
dt2 = load_dataset('csv', data_files = {'train':'/content/gdrive/MyDrive/Copy of cnn_train_class2.csv',
                                       'validation': '/content/gdrive/MyDrive/Copy of cnn_val_class2.csv',
                                       'test': '/content/gdrive/MyDrive/Copy of cnn_test_class2.csv'})

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d65b1ea57208c04c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dt2

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 44471
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 1539
    })
    test: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 1326
    })
})

In [7]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [8]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [9]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [11]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_train2 = dt2['train'].map(preprocess_function, batched=True)
tokenized_val2 =dt2['validation'].map(preprocess_function, batched=True)
tokenized_test2 =dt2['test'].map(preprocess_function, batched=True)


  0%|          | 0/45 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [13]:
tokenized_train2=tokenized_train2.remove_columns(dt2['train'].column_names)
tokenized_test2=tokenized_test2.remove_columns(dt2['test'].column_names)
tokenized_val2=tokenized_val2.remove_columns(dt2['validation'].column_names)

features = [tokenized_train2[i] for i in range(2)]

In [14]:
tf_train2_dataset = model.prepare_tf_dataset(
    tokenized_train2,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8)
  
tf_eval2_dataset = model.prepare_tf_dataset(
    tokenized_val2,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8)
  

num_train_epochs = 8
num_train_steps = len(tf_train2_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01)


model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [15]:
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-test", tokenizer=tokenizer
)

model.fit(
    tf_train2_dataset, validation_data=tf_eval2_dataset, callbacks=[callback], epochs=8
)

Cloning https://huggingface.co/lyhhhhhh/mt5-small-finetuned-test into local empty directory.


Epoch 1/8
5558/5558 [==============================] - ETA: 0s - loss: 4.4693

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


5558/5558 [==============================] - 624s 108ms/step - loss: 4.4693 - val_loss: 2.3874
Epoch 2/8
5558/5558 [==============================] - 575s 103ms/step - loss: 3.0670 - val_loss: 2.1557
Epoch 3/8
5558/5558 [==============================] - 575s 103ms/step - loss: 2.7416 - val_loss: 2.0547
Epoch 4/8
5558/5558 [==============================] - 575s 103ms/step - loss: 2.5824 - val_loss: 2.0089
Epoch 5/8
5558/5558 [==============================] - 569s 102ms/step - loss: 2.4922 - val_loss: 1.9654
Epoch 6/8
5558/5558 [==============================] - 569s 102ms/step - loss: 2.4299 - val_loss: 1.9344
Epoch 7/8
5558/5558 [==============================] - 569s 102ms/step - loss: 2.3906 - val_loss: 1.9255
Epoch 8/8
5558/5558 [==============================] - 570s 103ms/step - loss: 2.3668 - val_loss: 1.9101


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.33k/2.07G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/lyhhhhhh/mt5-small-finetuned-test
   0ef8e48..7d51496  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/lyhhhhhh/mt5-small-finetuned-test
   0ef8e48..7d51496  main -> main



In [19]:
model_class2 = model

In [21]:
from tqdm import tqdm
import numpy as np

generation_data2_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model_class2, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate2_dataset = model_class2.prepare_tf_dataset(
    tokenized_val2,
    collate_fn=generation_data2_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [22]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model_class2.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )

In [24]:
all_preds2 = []
all_labels2 = []
for batch, labels in tqdm(tf_generate2_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds2 = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels2 = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds2.extend(decoded_preds2)
    all_labels2.extend(decoded_labels2)

100%|██████████| 192/192 [00:17<00:00, 10.79it/s]


In [26]:
rouge_score = evaluate.load("rouge")
result2 = rouge_score.compute(
    predictions=decoded_preds2, references=decoded_labels2, use_stemmer=True
)

## rough score for validation data
result2 = {key: value* 100 for key, value in result2.items()}
{k: round(v, 4) for k, v in result2.items()}

{'rouge1': 33.9453, 'rouge2': 22.3662, 'rougeL': 30.1175, 'rougeLsum': 31.8422}

In [27]:
tf_generate2_test_dataset = model_class2.prepare_tf_dataset(
    tokenized_test2,
    collate_fn=generation_data2_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [59]:
all_preds2_test = []
all_labels2_test = []
decoded_preds2 = []
decoded_labels2 = []
for batch, labels in tqdm(tf_generate2_test_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds2_test = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels2_test = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds2_test.extend(decoded_preds2_test)
    all_labels2_test.extend(decoded_labels2_test)

100%|██████████| 165/165 [00:14<00:00, 11.34it/s]


In [48]:
import pandas as pd
df_class2_test = pd.DataFrame(all_preds2_test)
df_class2_test.to_csv('pred_test_class2.csv')

In [72]:
df_test = pd.DataFrame(dt2['test'])
df_test_new = df_test.iloc[:1320, 1:]
df_test_new.head()

,article,highlights,class
0,"(CNN)For the first time in eight years, a TV l...","Bob Barker returned to host ""The Price Is Righ...",2
1,(CNN)A year ago Bloomberg published a story wi...,"LZ: Indiana law pushing back LGBT rights, and ...",2
2,"(CNN)As goes Walmart, so goes the nation? Ever...",While Republican Gov. Asa Hutchinson was weigh...,2
3,Los Angeles (CNN)It's more than just one state...,Americans paid more for some fruits and vegeta...,2
4,(CNN)A Connecticut teen who has been forced to...,Judge won't allow teen leave hospital before h...,2


In [74]:
df_test_new['summary'] = df_class2_test
df_test_new.head()

,article,highlights,class,summary
0,"(CNN)For the first time in eight years, a TV l...","Bob Barker returned to host ""The Price Is Righ...",2,Bob Barker hosted the TV game show for 35 year...
1,(CNN)A year ago Bloomberg published a story wi...,"LZ: Indiana law pushing back LGBT rights, and ...",2,"Mike Pence, a Koch Favorite, Mulls 2016 Run fo..."
2,"(CNN)As goes Walmart, so goes the nation? Ever...",While Republican Gov. Asa Hutchinson was weigh...,2,"Walmart says it would boost pay for about 500,..."
3,Los Angeles (CNN)It's more than just one state...,Americans paid more for some fruits and vegeta...,2,"California drought hurts rest of the union, to..."
4,(CNN)A Connecticut teen who has been forced to...,Judge won't allow teen leave hospital before h...,2,"NEW: Cassandra C., 17, is in remission after n..."


In [75]:
df_test_new.to_csv('test_class2_final.csv')

In [76]:
!cp pred_test_class2.csv "gdrive/My Drive"
!cp test_class2_final.csv "gdrive/My Drive"

In [29]:
rouge_score = evaluate.load("rouge")
result2_test = rouge_score.compute(
    predictions=decoded_preds2_test, references=decoded_labels2_test, use_stemmer=True
)

## rough score for test data 
result2_test = {key: value* 100 for key, value in result2_test.items()}
{k: round(v, 4) for k, v in result2_test.items()}

{'rouge1': 35.6826, 'rouge2': 15.1027, 'rougeL': 31.5934, 'rougeLsum': 31.4489}

In [77]:
model_class2.push_to_hub("lyhhhhhh/mt5-small-finetuned-test-class2")